In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join('..')))

from src.loading import DataLoader
from src.cleaning import DataCleaner
from src.weighting import DataWeighter
from src.similarity_gpu import NetworkBuilder

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
dl = DataLoader()
BP, CC, MF, HPO, DepthBP, DepthCC, DepthMF = dl.load_and_prepare_data()

Loading DATA from ../data/raw/
[OK] BP data loaded: (5183, 9873)
[OK] CC data loaded: (5183, 1478)
[OK] MF data loaded: (5183, 3258)
[OK] HPO data loaded: (5183, 10185)
[OK] DepthBP data loaded: (1, 9566)
[OK] DepthCC data loaded: (1, 1453)
[OK] DepthMF data loaded: (1, 3157)


In [8]:
dc = DataCleaner(BP, CC, MF, HPO, DepthBP, DepthCC, DepthMF)
BP, CC, MF, HPO, DepthBP, DepthCC, DepthMF = dc.clean_all()

[OK] BP data cleaned: (4153, 6115)
[OK] CC data cleaned: (4153, 880)
[OK] MF data cleaned: (4153, 1335)
[OK] HPO data cleaned: (4153, 6342)
[OK] DepthBP data cleaned: (1, 6013)
[OK] DepthCC data cleaned: (1, 867)
[OK] DepthMF data cleaned: (1, 1315)


In [9]:
dw = DataWeighter(BP, CC, MF, HPO, DepthBP, DepthCC, DepthMF)
BP_tfidf, CC_tfidf, MF_tfidf, HPO_tfidf = dw.transform_all()

=== Depth-Weighted TF-IDF ===

[BP] Applying depth weighting...
  [DEPTH] 102/6115 terms missing from depth file -> imputed with median depth (5.0)
[BP] Applying TF-IDF...
[OK] BP transformed: (4153, 6115)

[CC] Applying depth weighting...
  [DEPTH] 13/880 terms missing from depth file -> imputed with median depth (4.0)
[CC] Applying TF-IDF...
[OK] CC transformed: (4153, 880)

[MF] Applying depth weighting...
  [DEPTH] 20/1335 terms missing from depth file -> imputed with median depth (4.0)
[MF] Applying TF-IDF...
[OK] MF transformed: (4153, 1335)

[HPO] No depth file available — applying plain TF-IDF...
[OK] HPO transformed: (4153, 6342)

[SAVED] All TF-IDF matrices written to ../data/processed/


In [ ]:
snf_builder = NetworkBuilder(BP_tfidf, CC_tfidf, MF_tfidf, HPO_tfidf)
fused = snf_builder.build_and_fuse()

=== Similarity Network Fusion ===

[BP] Feature matrix: (4153, 6115)
[CC] Feature matrix: (4153, 880)
[MF] Feature matrix: (4153, 1335)
[HPO] Feature matrix: (4153, 6342)

[SNF] Building affinity matrices (K=20, mu=0.5, metric=cosine)...
